# Convert raw txt output to JSON

Columns in txt are:

```bifurcation ID, number of bounces, pr, ptheta, pphi, energy, rocking number, in phase/out of phase, period```

In [1]:
import json
import re

data = []
bids = []
energies = []
with open('./bifurcation.txt', 'r') as infile:
    for line in infile:
        line = line.strip()
        (bid, n, pr, ptheta, pphi, E, rocking, phase, period) = line.split(' ')
        bids.append(bid)
        data.append({ 'id':int(bid), 'n':int(n), 'pr':float(pr),
                     'ptheta':float(ptheta), 'pphi':float(pphi), 'E':float(E),
                     'rocking':int(rocking), 'phase':int(phase), 'period':float(period)})
        energies.append(float(E))

with open('./bifurcation.json', 'w') as outfile:  
    json.dump(data, outfile)
print('done')

done
